### Problemes a resoudre dans ce DATASET

    1.Valeurs manquantes (NaN)
    2.  Doublons (clients en double)
    3.  Formats incohérents (dates, téléphones)
    4.  Casse incohérente (paris, PARIS, Paris)
    5.  Espaces en trop
    6.  Valeurs aberrantes (âge -5, âge 156)
    7.  Emails invalides
    8.  Catégories mal orthographiées
    9.  Booléens incohérents (oui/OUI/1/Oui)  
    10.  Montants négatifs ou irréalistes

In [23]:
# Chargement des données et exploration
import pandas as pd
import numpy as np

df = pd.read_csv('../data/raw/clients_ecommerce_raw.csv')

print(df.head()) 
print(f"lignes et colonnes : {df.shape}")


   client_id             nom                      email       telephone  \
0          1   Martin Dupont    martin.dupont@gmail.com      0612345678   
1          2    marie claire      marie.claire@yahoo.fr  06 23 45 67 89   
2          3      JEAN PETIT       JEAN.PETIT@GMAIL.COM    +33634567890   
3          4   Sophie Martin  sophie.martin@hotmail.com      0645678901   
4          5  Pierre  Durand    pierre.durand@gmail.com  06.56.78.90.12   

     age      ville date_inscription  montant_achats  nb_commandes  \
0   25.0      Paris       2023-01-15          150.50           3.0   
1   34.0      paris       15/02/2023          230.00           5.0   
2   -5.0       LYON       2023-03-20          -50.00           0.0   
3   45.0  Marseille       20-04-2023           89.99           2.0   
4  156.0       Lyon       2023/05/10         1500.00          25.0   

  categorie_preferee newsletter  
0       Électronique        oui  
1               Mode        OUI  
2       electronique      

In [24]:
# Audit
# creation d'un rapport :

def audit(df):
    "Rapport du dataset: "
    rapport = pd.DataFrame({
        'Valeur Manquante' : df.isnull().sum(),
        'Doublons' : df.duplicated().sum(),
        'Types correct': df.dtypes,
        'Valeur unique': df.nunique(),
    })
    return rapport

audit(df)

,Valeur Manquante,Doublons,Types correct,Valeur unique
client_id,0,1,int64,29
nom,2,1,object,27
email,2,1,object,27
telephone,3,1,object,26
age,4,1,float64,24
ville,2,1,object,27
date_inscription,1,1,object,28
montant_achats,4,1,float64,25
nb_commandes,4,1,float64,16
categorie_preferee,3,1,object,9


In [25]:
# identification des patterns

print(f"check les valeurs : \n {df.describe()}\n")
print(f"check les format incohérent (NOM): \n {df['nom'].unique()}\n")
print(f"check les format incohérent (EMAIL): \n {df['email'].unique()}\n")
print(f"check les format incohérent (TELEPHONE): \n {df['telephone'].unique()}\n")
print(f"check les format incohérent (VILLE): \n {df['ville'].unique()}\n")
print(f"check les format incohérent (DATA INSCRIPTION): \n {df['date_inscription'].unique()}\n")
print(f"check les format incohérent (CATEGORIE): \n {df['categorie_preferee'].unique()}\n")
print(f"check les format incohérent (NEWSLETTER): \n {df['newsletter'].unique()}\n")


check les valeurs : 
        client_id         age  montant_achats  nb_commandes
count  30.000000   26.000000       26.000000     26.000000
mean   15.033333   40.192308    38765.252308     11.846154
std     9.140600   26.561655   196054.458596     28.706365
min     1.000000   -5.000000      -50.000000      0.000000
25%     7.250000   29.250000      126.212500      3.000000
50%    14.500000   35.500000      232.250000      5.000000
75%    22.750000   44.750000      420.150000      8.750000
max    30.000000  156.000000   999999.990000    150.000000

check les format incohérent (NOM): 
 ['Martin Dupont' 'marie claire' 'JEAN PETIT' 'Sophie Martin'
 'Pierre  Durand' nan 'Lucas Bernard' 'emma WILSON' 'Thomas Leroy'
 'Julie Moreau' 'Antoine Roux' 'Camille Fournier' 'Nicolas Girard'
 'Laura Bonnet' 'Maxime Dupuis' 'Pauline Lambert' 'Vincent Faure'
 'Céline André' 'Romain Mercier' 'Amélie Simon' 'Florian Michel'
 'Charlotte Garcia' 'Julien Martinez' 'Manon Lopez' 'Alexandre Gonzalez'
 'Océane S

Patterns identifiés
1. client_id
   - Doublon : id 2 apparaît 2 fois

2. nom
   - Casse incohérente (MAJUSCULES, minuscules, Mixte)
   - Doubles espaces
   - Valeurs manquantes (NaN)

3. email
   - Majuscules dans certains emails
   - Emails invalides (sans @)
   - Valeurs manquantes

4. telephone
   - Formats multiples : 06, +33, espaces, points, tirets
   - Valeurs manquantes

5. ville
   - Casse incohérente (paris, PARIS, Paris)

6. date_inscription
   - Formats différents : JJ/MM/AAAA vs AAAA-MM-JJ

7. categorie
   - Variations : premium, Premium, VIP, standard

8. montant_total
   - Valeurs négatives
   - Outliers extrêmes (999999)

9. newsletter
   - Formats multiples : oui/non, O/N, True/False, 1/0

10. age
    - Valeurs manquantes
    - Valeurs impossibles (150 ans, négatif)

11. nb_commandes
    - Valeurs manquantes
    - Possibles valeurs négatives

### Traitement des valeurs Manquantes

In [26]:
# Afficher état initial
print(f"\nÉTAT INITIAL : {df.isnull().sum().sum()} valeurs manquantes")
print(f"Lignes : {len(df)}")


ÉTAT INITIAL : 28 valeurs manquantes
Lignes : 30


In [27]:
from datetime import datetime

df['nom'] = df['nom'].fillna('Client Inconnu') # Remplacer les nom par Clients Inconnu
df['email'] = df.apply(lambda x: f"client_{x['client_id']}@temp.com" if pd.isna(x['email']) else x['email'], axis=1) # Creation d'un email
df['telephone'] = df['telephone'].fillna('Non renseigné') # Remplacer par Non renseigné
df['age'] = df['age'].fillna(df['age'].median()) # Replacer par la median des autres âge
df['ville'] = df['ville'].fillna(df['ville'].mode()[0]) # Prendre la ville la plus utilisé
df['date_inscription'] = df['date_inscription'].fillna(datetime.now().strftime('%Y-%m-%d')) # Remplacer par la date du jour
df['montant_achats'] = df['montant_achats'].fillna(0) # Remplacer par 0 (new client)
df['nb_commandes'] = df['nb_commandes'].fillna(0)# Idem
df['categorie_preferee'] = df['categorie_preferee'].fillna('Non définie') # Remplacer par Non définie
df['newsletter'] = df['newsletter'].fillna('Non') # Remplacer par Non

print(f"NaN après : {df.isnull().sum().sum()}")


NaN après : 0


### Détection et suppression des doublons

In [28]:
print(f"Lignes avant : {len(df)}")

# Analyser les doublons
print(f"  doublons Parfaits : {df.duplicated().sum()}")
print(f"  doublons Client_id : {df.duplicated(subset=['client_id']).sum()}")
print(f"  doublons Email : {df.duplicated(subset=['email']).sum()}")

# Supprimer (garder première occurrence)
df = df.drop_duplicates(keep='first')

print(f"\nLignes après : {len(df)}")
print("Doublons supprimés !")

print("Apres :")
print(f"  doublons Parfaits : {df.duplicated().sum()}")
print(f"  doublons Client_id : {df.duplicated(subset=['client_id']).sum()}")
print(f"  doublons Email : {df.duplicated(subset=['email']).sum()}")




Lignes avant : 30
  doublons Parfaits : 1
  doublons Client_id : 1
  doublons Email : 1

Lignes après : 29
Doublons supprimés !
Apres :
  doublons Parfaits : 0
  doublons Client_id : 0
  doublons Email : 0
